# Google Civic API
https://developers.google.com/civic-information/docs/using_api

Notes:
* voterinfo returns a subset of documented info. I assume that Google is still populating
* normalizedInput does *NOT* validate addresses


In [60]:
import json
import os
import pprint
import requests

from IPython.display import HTML, Markdown, display
import tabulate

In [41]:
# get a key at https://console.developers.google.com/apis/credentials
creds_fn = os.path.expanduser("~/.credentials/google-civic-api.json")
google_api_key = json.load(open(creds_fn))["api_key"]

In [42]:
google_session = requests.Session()
google_session.params.update({"key": google_api_key})

## Elections

In [43]:
elections_resp = google_session.get("https://www.googleapis.com/civicinfo/v2/elections")
elections_resp.raise_for_status()
elections = elections_resp.json()

In [44]:
elections.keys()

dict_keys(['elections', 'kind'])

In [45]:
len(elections["elections"])

4

In [46]:
elections["elections"]

[{'id': '2000',
  'name': 'VIP Test Election',
  'electionDay': '2021-06-06',
  'ocdDivisionId': 'ocd-division/country:us'},
 {'id': '5016',
  'name': 'Delaware State Primary Election',
  'electionDay': '2020-09-15',
  'ocdDivisionId': 'ocd-division/country:us/state:de'},
 {'id': '5018',
  'name': 'Rhode Island State Primary Election',
  'electionDay': '2020-09-08',
  'ocdDivisionId': 'ocd-division/country:us/state:ri'},
 {'id': '7000',
  'name': 'US 2020 General Election',
  'electionDay': '2020-11-03',
  'ocdDivisionId': 'ocd-division/country:us'}]

In [48]:
election_id = 2000

## VoterInfo

In [101]:
voterinfo_resp = google_session.get("https://www.googleapis.com/civicinfo/v2/voterinfo",
                                    params={"address":"15032 Sussex St., 94131",
                                            "electionId": election_id
                                           })
voterinfo_resp.raise_for_status()
voterinfo = voterinfo_resp.json()

In [102]:
# response keys; responses may not have all keys
voterinfo_keys = [
    'kind', 'election', 'otherElections', 'normalizedInput', 'mailOnly',
    'pollingLocations', 'earlyVotingSites', 'dropOffLocations', 'contests', 'state' ]

In [103]:
def vi_markdown(vi, vik):
    d = voterinfo.get(vik, None)
    if d is None:
        t = "None"
    elif isinstance(d, list):
        t = f"list; {len(d)} elements"
    elif isinstance(d, dict):
        t = f"dict; {len(d.keys())} keys"
    else:
        t = type(d).__name__
    s = f"#### {vik} ({t})\n\n"
    if d is not None:
        s += "```\n" + pprint.pformat(d) + "\n```\n"
    return s

def vi_display(vi, vik):
    display(Markdown(vi_markdown(vi, vik)))

In [104]:
vi_display(voterinfo, "kind")

#### kind (str)

```
'civicinfo#voterInfoResponse'
```


In [105]:
vi_display(voterinfo, "election")

#### election (dict; 4 keys)

```
{'electionDay': '2021-06-06',
 'id': '2000',
 'name': 'VIP Test Election',
 'ocdDivisionId': 'ocd-division/country:us'}
```


In [106]:
vi_display(voterinfo, "otherElections")

#### otherElections (None)



In [107]:
vi_display(voterinfo, "normalizedInput")

#### normalizedInput (dict; 4 keys)

```
{'city': 'San Francisco',
 'line1': '15032 Sussex Street',
 'state': 'CA',
 'zip': '94131'}
```


In [98]:
vi_display(voterinfo, "mailOnly")

#### mailOnly (None)



In [93]:
vi_display(voterinfo, "pollingLocations")

#### pollingLocations (list; 1 elements)

```
[{'address': {'city': 'San Francisco',
              'line1': '742 Chenery St',
              'locationName': 'Garage',
              'state': 'CA',
              'zip': '94131-2907'},
  'notes': 'Btw Lippard & Brompton Ave',
  'pollingHours': '07:00-20:00',
  'sources': [{'name': 'Voting Information Project', 'official': True}]}]
```


In [94]:
vi_display(voterinfo, "earlyVotingSites")

#### earlyVotingSites (None)



In [95]:
vi_display(voterinfo, "dropOffLocations")

#### dropOffLocations (None)



In [97]:
vi_display(voterinfo, "contests")

#### contests (list; 20 elements)

```
[{'candidates': [{'name': 'Nancy Pelosi', 'party': 'Democratic'},
                 {'candidateUrl': 'http://www.johndennisforcongress.com/',
                  'channels': [{'id': 'https://www.facebook.com/johndennis2010',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/johndennis2012',
                                'type': 'Twitter'}],
                  'name': 'John Dennis',
                  'party': 'Republican'}],
  'district': {'id': 'ocd-division/country:us/state:ca/cd:12',
               'name': "California's 12th congressional district",
               'scope': 'congressional'},
  'level': ['country'],
  'office': 'US House - District 12',
  'roles': ['legislatorLowerBody'],
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'General'},
 {'candidates': [{'channels': [{'id': 'https://www.facebook.com/jerrybrown',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/JerryBrownGov',
                                'type': 'Twitter'},
                               {'id': 'https://www.youtube.com/user/JerryBrown2010',
                                'type': 'YouTube'}],
                  'name': 'Edmund G. "Jerry" Brown',
                  'party': 'Democratic'},
                 {'candidateUrl': 'http://www.neelkashkari.com/',
                  'channels': [{'id': 'https://www.facebook.com/neelkashkari',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/neelkashkari',
                                'type': 'Twitter'},
                               {'id': 'https://www.youtube.com/channel/UCGhIiUj05N8mlUNeiKbYfsA',
                                'type': 'YouTube'}],
                  'name': 'Neel Kashkari',
                  'party': 'Republican'}],
  'district': {'id': 'ocd-division/country:us/state:ca',
               'name': 'California',
               'scope': 'statewide'},
  'level': ['administrativeArea1'],
  'office': 'Governor',
  'roles': ['headOfGovernment'],
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'General'},
 {'candidates': [{'candidateUrl': 'http://www.ronnehring.com/',
                  'channels': [{'id': 'https://www.facebook.com/RonNehringCA',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/RonNehring',
                                'type': 'Twitter'},
                               {'id': 'https://www.youtube.com/channel/UCFGwCF7iZWLP-j0dFWORvGQ',
                                'type': 'YouTube'}],
                  'name': 'Ron Nehring',
                  'party': 'Republican'},
                 {'candidateUrl': 'http://www.gavinnewsom.com/',
                  'channels': [{'id': 'https://www.facebook.com/GavinNewsom',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/GavinNewsom',
                                'type': 'Twitter'},
                               {'id': 'https://www.youtube.com/user/NewsomforCalifornia',
                                'type': 'YouTube'}],
                  'email': 'gavin@gavinnewsom.com',
                  'name': 'Gavin Newsom',
                  'party': 'Democratic',
                  'phone': '(415) 963-9240'}],
  'district': {'id': 'ocd-division/country:us/state:ca',
               'name': 'California',
               'scope': 'statewide'},
  'level': ['administrativeArea1'],
  'office': 'Lieutenant Governor',
  'roles': ['deputyHeadOfGovernment'],
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'General'},
 {'candidates': [{'candidateUrl': 'http://www.electjohnchiang.com/',
                  'channels': [{'id': 'https://www.facebook.com/22930503694',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/johnchiang4ca',
                                'type': 'Twitter'},
                               {'id': 'https://www.youtube.com/user/JohnChiang4CA',
                                'type': 'YouTube'}],
                  'email': 'ElectJohnChiang@gmail.com',
                  'name': 'John Chiang',
                  'party': 'Democratic'},
                 {'candidateUrl': 'http://www.gregconlon.com/',
                  'channels': [{'id': 'https://www.facebook.com/gregconlonforstatetreasurer',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/GregConlon',
                                'type': 'Twitter'},
                               {'id': 'https://www.youtube.com/channel/UClllihMKVeAvNVsdYmo-frg',
                                'type': 'YouTube'}],
                  'email': 'gregconlonforstatetreasurer@yahoo.com',
                  'name': 'Greg Conlon',
                  'party': 'Republican',
                  'phone': '916-850-2782'}],
  'district': {'id': 'ocd-division/country:us/state:ca',
               'name': 'California',
               'scope': 'statewide'},
  'level': ['administrativeArea1'],
  'office': 'Treasurer',
  'roles': ['governmentOfficer'],
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'General'},
 {'candidates': [{'candidateUrl': 'http://www.rongold.org/',
                  'channels': [{'id': 'https://www.youtube.com/channel/UC1Od8iX56vySEzXQJ3O8SMw',
                                'type': 'YouTube'}],
                  'name': 'Ronald Gold',
                  'party': 'Republican'},
                 {'candidateUrl': 'http://kamalaharris.org/',
                  'channels': [{'id': 'https://www.facebook.com/KamalaHarris',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/KamalaHarris',
                                'type': 'Twitter'},
                               {'id': 'https://www.youtube.com/user/kamalaharrisdotorg',
                                'type': 'YouTube'}],
                  'email': 'info@kamalaharris.org',
                  'name': 'Kamala D. Harris',
                  'party': 'Democratic',
                  'phone': '(415) 364-8784'}],
  'district': {'id': 'ocd-division/country:us/state:ca',
               'name': 'California',
               'scope': 'statewide'},
  'level': ['administrativeArea1'],
  'office': 'Attorney General',
  'roles': ['governmentOfficer'],
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'General'},
 {'candidates': [{'candidateUrl': 'http://www.padilla4sofs.com/',
                  'channels': [{'id': 'https://www.facebook.com/senator.padilla',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/SenAlexPadilla',
                                'type': 'Twitter'},
                               {'id': 'https://www.youtube.com/channel/UC8iDXwFo62EvFaNIrQln0vA',
                                'type': 'YouTube'}],
                  'name': 'Alex Padilla',
                  'party': 'Democratic',
                  'phone': '818-253-9140'},
                 {'candidateUrl': 'http://www.petesos.com/',
                  'channels': [{'id': 'https://www.facebook.com/PeteSOS',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/Pete4SOS',
                                'type': 'Twitter'}],
                  'email': 'campaign@petesos.com',
                  'name': 'Pete Peterson',
                  'party': 'Republican',
                  'phone': '323-450-7536'}],
  'district': {'id': 'ocd-division/country:us/state:ca',
               'name': 'California',
               'scope': 'statewide'},
  'level': ['administrativeArea1'],
  'office': 'Secretary of State',
  'roles': ['governmentOfficer'],
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'General'},
 {'candidates': [{'candidateUrl': 'http://www.tomtorlakson.com/',
                  'channels': [{'id': 'https://www.facebook.com/torlakson',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/TomTorlakson',
                                'type': 'Twitter'},
                               {'id': 'https://www.youtube.com/user/tomtorlakson',
                                'type': 'YouTube'}],
                  'name': 'Tom Torlakson',
                  'party': 'Nonpartisan',
                  'phone': '925-386-6774'},
                 {'candidateUrl': 'http://www.marshalltuck.com/',
                  'channels': [{'id': 'https://www.facebook.com/TuckforCalifornia',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/MarshallTuck',
                                'type': 'Twitter'},
                               {'id': 'https://www.youtube.com/channel/UCAd4z6Ea8kBxxI-O0f1SbiA',
                                'type': 'YouTube'}],
                  'name': 'Marshall Tuck',
                  'party': 'Nonpartisan'}],
  'district': {'id': 'ocd-division/country:us/state:ca',
               'name': 'California',
               'scope': 'statewide'},
  'level': ['administrativeArea1'],
  'office': 'Superintendent of Public Instruction',
  'roles': ['governmentOfficer'],
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'General'},
 {'candidates': [{'candidateUrl': 'http://www.tedgaines.com/',
                  'channels': [{'id': 'https://www.facebook.com/ted.gaines',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/TedGaines',
                                'type': 'Twitter'},
                               {'id': 'https://www.youtube.com/user/GainesForIC2014',
                                'type': 'YouTube'}],
                  'name': 'Ted Gaines',
                  'party': 'Republican',
                  'phone': '916-221-5770'},
                 {'candidateUrl': 'http://www.davejones2014.com/',
                  'channels': [{'id': 'https://www.facebook.com/239029093423',
                                'type': 'Facebook'}],
                  'email': 'TeamDaveJones@gmail.com',
                  'name': 'Dave Jones',
                  'party': 'Democratic',
                  'phone': '(916) 349-4236'}],
  'district': {'id': 'ocd-division/country:us/state:ca',
               'name': 'California',
               'scope': 'statewide'},
  'level': ['administrativeArea1'],
  'office': 'Insurance Commissioner',
  'roles': ['governmentOfficer'],
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'General'},
 {'candidates': [{'candidateUrl': 'http://www.ashleyforca.com/',
                  'channels': [{'id': 'https://www.facebook.com/ashleyswearengin',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/Ashley4CA',
                                'type': 'Twitter'},
                               {'id': 'https://www.youtube.com/channel/UC9Tc_GLpw5X7t_gg-Q51eKw',
                                'type': 'YouTube'}],
                  'name': 'Ashley Swearengin',
                  'party': 'Republican',
                  'phone': '(559) 565-5066'},
                 {'candidateUrl': 'http://bettyyee.com/',
                  'channels': [{'id': 'https://www.facebook.com/BettyYee2014',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/bettyyee2014',
                                'type': 'Twitter'}],
                  'email': 'info@bettyyee.com',
                  'name': 'Betty T. Yee',
                  'party': 'Democratic',
                  'phone': '(415) 692-3556'}],
  'district': {'id': 'ocd-division/country:us/state:ca',
               'name': 'California',
               'scope': 'statewide'},
  'level': ['administrativeArea1'],
  'office': 'Controller',
  'roles': ['governmentOfficer'],
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'General'},
 {'candidates': [{'name': 'Goodwin Liu', 'party': 'Nonpartisan'},
                 {'name': 'Kathryn Mickle Werdegar', 'party': 'Nonpartisan'},
                 {'name': 'Mariano-Florentino Cuellar',
                  'party': 'Nonpartisan'}],
  'district': {'id': 'ocd-division/country:us/state:ca',
               'name': 'California',
               'scope': 'statewide'},
  'level': ['administrativeArea1'],
  'office': 'Supreme Court Associate Justice (Retention)',
  'roles': ['highestCourtJudge'],
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'General'},
 {'candidates': [{'candidateUrl': 'http://www.davidcampossf.com/',
                  'channels': [{'id': 'https://www.facebook.com/davidcampossf',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/davidcampossf',
                                'type': 'Twitter'}],
                  'name': 'David Campos',
                  'party': 'Democratic'},
                 {'candidateUrl': 'http://www.votedavidchiu.com/',
                  'channels': [{'id': 'https://www.facebook.com/davidchiu.sf',
                                'type': 'Facebook'},
                               {'id': 'https://twitter.com/davidchiu',
                                'type': 'Twitter'}],
                  'name': 'David Chiu',
                  'party': 'Democratic'}],
  'district': {'id': 'ocd-division/country:us/state:ca/sldl:17',
               'name': 'California Assembly district 17',
               'scope': 'stateLower'},
  'level': ['administrativeArea1'],
  'office': 'State Assembly -  District 17',
  'roles': ['legislatorLowerBody'],
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'General'},
 {'candidates': [{'name': 'Carmen Chu', 'party': 'Nonpartisan'}],
  'district': {'id': 'ocd-division/country:us/state:ca/county:san_francisco',
               'name': 'San Francisco County',
               'scope': 'countywide'},
  'level': ['administrativeArea2'],
  'office': 'Assessor/Recorder',
  'roles': ['governmentOfficer'],
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'General'},
 {'candidates': [{'name': 'Jeff Adachi', 'party': 'Nonpartisan'}],
  'district': {'id': 'ocd-division/country:us/state:ca/county:san_francisco',
               'name': 'San Francisco County',
               'scope': 'countywide'},
  'level': ['administrativeArea2'],
  'office': 'Public Defender',
  'roles': ['governmentOfficer'],
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'General'},
 {'candidates': [{'name': 'George Davis', 'party': 'Nonpartisan'},
                 {'channels': [{'id': 'https://www.facebook.com/johnnultysupervisor',
                                'type': 'Facebook'}],
                  'name': 'John Nulty',
                  'party': 'Nonpartisan'},
                 {'channels': [{'id': 'https://www.facebook.com/petrelis4supe8',
                                'type': 'Facebook'}],
                  'name': 'Michael Petrelis',
                  'party': 'Nonpartisan'},
                 {'name': 'Scott Wiener', 'party': 'Nonpartisan'},
                 {'name': 'Tom Wayne Basso', 'party': 'Nonpartisan'}],
  'district': {'id': 'ocd-division/country:us/state:ca/county:san_francisco/council_district:8',
               'name': 'San Francisco CA city-county council district 8',
               'scope': 'countyCouncil'},
  'level': ['administrativeArea2'],
  'office': 'Supervisor District 8',
  'roles': ['legislatorUpperBody'],
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'General'},
 {'district': {'id': 'ocd-division/country:us/state:ca',
               'name': 'California',
               'scope': 'statewide'},
  'referendumSubtitle': 'Water Bond. Funding for Water Quality, Supply, '
                        'Treatment, and Storage Projects.',
  'referendumTitle': 'Proposition 1',
  'referendumUrl': 'http://vig.cdn.sos.ca.gov/2014/general/en/pdf/proposition-1-title-summary-analysis-v2.pdf',
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'Referendum'},
 {'district': {'id': 'ocd-division/country:us/state:ca',
               'name': 'California',
               'scope': 'statewide'},
  'referendumSubtitle': 'State Budget. Budget Stabilization Account. '
                        'Legislative Constitutional Amendment.',
  'referendumTitle': 'Proposition 2',
  'referendumUrl': 'http://vig.cdn.sos.ca.gov/2014/general/en/pdf/proposition-2-title-summary-analysis.pdf',
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'Referendum'},
 {'district': {'id': 'ocd-division/country:us/state:ca',
               'name': 'California',
               'scope': 'statewide'},
  'referendumSubtitle': 'Healthcare Insurance. Rate Changes. Initiative '
                        'Statute.',
  'referendumTitle': 'Proposition 45',
  'referendumUrl': 'http://vig.cdn.sos.ca.gov/2014/general/en/pdf/proposition-45-title-summary-analysis.pdf',
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'Referendum'},
 {'district': {'id': 'ocd-division/country:us/state:ca',
               'name': 'California',
               'scope': 'statewide'},
  'referendumSubtitle': 'Drug and Alcohol Testing of Doctors. Medical '
                        'Negligence Lawsuits. Initiative Statute.',
  'referendumTitle': 'Proposition 46',
  'referendumUrl': 'http://vig.cdn.sos.ca.gov/2014/general/en/pdf/proposition-46-title-summary-analysis.pdf',
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'Referendum'},
 {'district': {'id': 'ocd-division/country:us/state:ca',
               'name': 'California',
               'scope': 'statewide'},
  'referendumSubtitle': 'Criminal Sentences. Misdemeanor Penalties. Initiative '
                        'Statute.',
  'referendumTitle': 'Proposition 47',
  'referendumUrl': 'http://vig.cdn.sos.ca.gov/2014/general/en/pdf/proposition-47-title-summary-analysis.pdf',
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'Referendum'},
 {'district': {'id': 'ocd-division/country:us/state:ca',
               'name': 'California',
               'scope': 'statewide'},
  'referendumSubtitle': 'Indian Gaming Compacts. Referendum.',
  'referendumTitle': 'Proposition 48',
  'referendumUrl': 'http://vig.cdn.sos.ca.gov/2014/general/en/pdf/proposition-48-title-summary-analysis.pdf',
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'type': 'Referendum'}]
```


In [100]:
vi_display(voterinfo, "state")

#### state (list; 1 elements)

```
[{'electionAdministrationBody': {'ballotInfoUrl': 'https://www.sos.ca.gov/elections/polling-place/',
                                 'correspondenceAddress': {'city': 'Sacramento',
                                                           'line1': '1500 11th '
                                                                    'Street',
                                                           'state': 'California',
                                                           'zip': '95814-5701'},
                                 'electionInfoUrl': 'http://www.sos.ca.gov/elections',
                                 'name': 'California Secretary of State',
                                 'votingLocationFinderUrl': 'https://www.sos.ca.gov/elections/polling-place/'},
  'local_jurisdiction': {'electionAdministrationBody': {'electionInfoUrl': 'http://sfelections.org',
                                                        'electionOfficials': [{'emailAddress': 'sfvote@sfgov.org',
                                                                               'officePhoneNumber': '(415) '
                                                                                                    '554-4375'}],
                                                        'physicalAddress': {'city': 'SAN '
                                                                                    'FRANCISCO',
                                                                            'line1': 'CITY '
                                                                                     'HALL '
                                                                                     '- '
                                                                                     '1 '
                                                                                     'DR. '
                                                                                     'CARLTON '
                                                                                     'B '
                                                                                     'GOODLETT '
                                                                                     'PLACE, '
                                                                                     'ROOM '
                                                                                     '48',
                                                                            'state': 'CA',
                                                                            'zip': '94102-4635'}},
                         'name': 'San Francisco County',
                         'sources': [{'name': 'DemocracyWorks',
                                      'official': False}]},
  'name': 'California',
  'sources': [{'name': '', 'official': False}]}]
```


## Partial Queries

In [123]:
params={"address":"1 Sussex St., 94131", "electionId": election_id}
google_session.get("https://www.googleapis.com/civicinfo/v2/voterinfo",
                   params={**params, "fields": "pollingLocations,earlyVoteSites,dropOffLocations"}).json()

{'pollingLocations': [{'address': {'locationName': 'Garage',
    'line1': '742 Chenery St',
    'city': 'San Francisco',
    'state': 'CA',
    'zip': '94131-2907'},
   'notes': 'Btw Lippard & Brompton Ave',
   'pollingHours': '07:00-20:00',
   'sources': [{'name': 'Voting Information Project', 'official': True}]}]}